# Assignment 2
Previously in assignment9_1_notmnist.ipynb, we created a pickle with formatted datasets for training, development and testing on the notMNIST dataset.

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

#### 学习使用TensorFlow

In [2]:
# these are all the modules we'll be using later. Make sure you can import them
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [1]:
pwd

'C:\\Users\\HsuhuiChan\\mypython\\NLP_course'

#### First reload the data we generated in assignment9_1_notmnist.ipynb.
#### 重载数据

In [52]:
pickle_file = 'D:\\code_project\\datasource\\notMNIST\\notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
f.close()   
del save #hint: to help gc free up memory
print('Training set:',train_dataset.shape, train_labels.shape)
print('Validation set:', valid_dataset.shape, valid_labels.shape)
print('Test set:', test_dataset.shape, test_labels.shape)

Training set: (2000, 28, 28) (2000,)
Validation set: (100, 28, 28) (100,)
Test set: (100, 28, 28) (100,)


Reformat into a shape that's more adapted to the models we're going to train:

data as a flat matrix,
labels as float 1-hot encodings.

#### 数据矩阵reshape

In [53]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = labels.reshape((-1,1))
    labels = np.array((np.arange(num_labels)==labels)).astype(np.float32)
    return dataset, labels

In [54]:
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set:',train_dataset.shape, train_labels.shape)
print('Validation set:', valid_dataset.shape, valid_labels.shape)
print('Test set:', test_dataset.shape, test_labels.shape)

Training set: (2000, 784) (2000, 10)
Validation set: (100, 784) (100, 10)
Test set: (100, 784) (100, 10)


In [55]:
# 观看这个用法
aa = (np.arange(3)==[[1],[2],[0]])
print(aa)
np.array(aa).astype(int)

[[False  True False]
 [False False  True]
 [ True False False]]


array([[0, 1, 0],
       [0, 0, 1],
       [1, 0, 0]])

We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:

. First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

with graph.as_default():
    ...
    
. Then you can run the operations on this graph as many times as you want by calling session.run(), providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

with tf.Session(graph=graph) as session:
    ...
Let's load all the data into TensorFlow and build the computation graph corresponding to our training：

#### 用tensorflow训练简单的多项式回归

In [58]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 1000

graph = tf.Graph()
with graph.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels])
    )
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits)
    )
    
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases
    )
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases )
    

Let's run this computation and iterate:

In [59]:
num_steps = 101

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels,1)) 
            / predictions.shape[0])

with tf.Session(graph = graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy arrays.
        _, loss_i, predictions_i = session.run([optimizer, loss, train_prediction])
        if (step %10 == 0):
            print('Loss at step %d: %f' %(step, loss_i))
            print('Training accuracy: %.1f %%' % accuracy(predictions_i, train_labels[:train_subset,:]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph dependencies.
            print('Validation accuracy: %.1f %%' %accuracy(valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f %%' %accuracy(test_prediction.eval(), test_labels ))


Initialized
Loss at step 0: 15.729875
Training accuracy: 11.5 %
Validation accuracy: 10.0 %
Test accuracy: 13.0 %
Loss at step 10: 4.910938
Training accuracy: 46.8 %
Validation accuracy: 46.0 %
Test accuracy: 49.0 %
Loss at step 20: 3.361525
Training accuracy: 60.3 %
Validation accuracy: 56.0 %
Test accuracy: 61.0 %
Loss at step 30: 2.694463
Training accuracy: 66.0 %
Validation accuracy: 65.0 %
Test accuracy: 61.0 %
Loss at step 40: 2.304038
Training accuracy: 68.9 %
Validation accuracy: 66.0 %
Test accuracy: 62.0 %
Loss at step 50: 2.029300
Training accuracy: 71.3 %
Validation accuracy: 68.0 %
Test accuracy: 64.0 %
Loss at step 60: 1.820327
Training accuracy: 72.9 %
Validation accuracy: 68.0 %
Test accuracy: 64.0 %
Loss at step 70: 1.654011
Training accuracy: 74.0 %
Validation accuracy: 67.0 %
Test accuracy: 68.0 %
Loss at step 80: 1.518119
Training accuracy: 76.1 %
Validation accuracy: 67.0 %
Test accuracy: 70.0 %
Loss at step 90: 1.404334
Training accuracy: 78.3 %
Validation accurac

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a Placeholder node which will be fed actual data at every call of session.run().

#### 使用SGD训练,重新构建网络

In [62]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size* image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros(num_labels))
    
    #Training computation
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases
                                )

Let's run it:

In [64]:
num_steps = 101

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch
        batch_data = train_dataset[offset: (offset + batch_size), :]
        batch_labels = train_labels[offset: (offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, loss_i, train_perdiction_i = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step%10 == 0):
            print('Minibatch loss at step %d: %f' %(step, loss_i))
            print('Minibatch accuracy: %.1f %%' %accuracy(train_perdiction_i, batch_labels))
            print('Validation accuracy: %.1f %%' %accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f %%' %accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.763811
Minibatch accuracy: 3.9 %
Validation accuracy: 13.0 %
Minibatch loss at step 10: 5.062395
Minibatch accuracy: 43.0 %
Validation accuracy: 37.0 %
Minibatch loss at step 20: 4.370882
Minibatch accuracy: 53.9 %
Validation accuracy: 53.0 %
Minibatch loss at step 30: 2.543247
Minibatch accuracy: 64.8 %
Validation accuracy: 56.0 %
Minibatch loss at step 40: 2.870123
Minibatch accuracy: 69.5 %
Validation accuracy: 62.0 %
Minibatch loss at step 50: 2.167030
Minibatch accuracy: 71.9 %
Validation accuracy: 66.0 %
Minibatch loss at step 60: 2.503828
Minibatch accuracy: 72.7 %
Validation accuracy: 64.0 %
Minibatch loss at step 70: 1.791514
Minibatch accuracy: 75.0 %
Validation accuracy: 66.0 %
Minibatch loss at step 80: 1.983242
Minibatch accuracy: 69.5 %
Validation accuracy: 69.0 %
Minibatch loss at step 90: 2.368526
Minibatch accuracy: 74.2 %
Validation accuracy: 70.0 %
Minibatch loss at step 100: 1.870625
Minibatch accuracy: 75.0 %
Validation accu

# Problem
Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units nn.relu() and 1024 hidden nodes. This model should improve your validation / test accuracy.